# EventBasedPortfolioManager - A portfolio manager for strategies.




## 1 EventBasedBacktester - A interface for strategy class that takes in a dataframe, and then trades on the data.

1. Data Collection
   1. Abstracted to a general function
2. Trade Execution
   1. Transaction costs
   2. Set stop loss
   3. Manages position and quantity tracking for multiple securities
   4. Handles basic buy/sell order execution=
   5. Manages position entry/exit
   6. Records trade details including timestamp, price, quantity
3. Risk Management
   1. Tracks realized account balance
   2. Monitors unrealized positions
   3. Set maximum position size
4. Performance Analysis
   1. Calculates various performance metrics (Sharpe ratio, drawdown, etc.)
   2. Provides visualization of performance through multiple plots
   3. Tracks trade history and portfolio evolution

In [1]:
import logging

from scipy import stats

logger = logging.getLogger(__name__)

from dataclasses import dataclass
from datetime import datetime
from enum import Enum
from typing import Optional, Union, List
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import yfinance as yf

class OrderType(Enum):
    MARKET = "MARKET"
    # LIMIT = "LIMIT"
    # STOP = "STOP"

class OrderSide(Enum):
    BUY = "BUY"
    SELL = "SELL"

# A trade is an order that has been executed. Assume that all trades were executed immediately. 
@dataclass
class Trade:
    timestamp : datetime
    side: OrderSide
    type: OrderType
    quantity: float
    price: Optional[float] = None

class EventBasedBacktester:
    """
    Event based backtest for generic trading strategy.
    """
    def __init__(self, symbol, start, end, interval, capital, transaction_cost, verbose = True):
        self.symbol = symbol
        self.start = start
        self.end = end
        self.interval = interval
        self.data = self.prepare_data()
        """
        Trade settings
        """
        self.initial_capital = capital
        self.transaction_cost = transaction_cost

        """
        Performance values.
        """
        self.current_position = 0
        self.current_capital = capital
        self.trades : List[Trade] = []
        # self.orders : Queue[Order] = []
        """
        Miscellanous
        """
        self.minimum_balance = self.initial_capital * 0.1
        self.verbose = verbose
        self.stop_trading = False

    def prepare_data(self):
        """
        Ensure data is in correct format with proper column names
        """
        stock_data = yf.Ticker(self.symbol)
        data = stock_data.history(start=self.start, end=self.end, interval=self.interval).reset_index()
        
        # # Standardize column names to lowercase
        # data = hist_stock.rename(columns={
        #     'Open': 'open',
        #     'High': 'high',
        #     'Low': 'low',
        #     'Close': 'close',
        #     'Volume': 'volume',
        #     'Date': 'datetime'
        # }).reset_index()
        # print(data)
        # Add timestamp column
        data['timestamp'] = data['Date'].apply(lambda x: int(x.timestamp() * 1000))
        data = data.sort_values('timestamp', ascending=True).reset_index(drop=True)
        
        # Calculate log returns
        data['logreturns_close'] = np.log(data['Close'] / data['Close'].shift(1))
        
        return data
    """
    Strategy Function - to override strategy() method.
    """
    def strategy(self, timestamp):
        # Should use the execute_order() method to execute trades.
        raise NotImplementedError("Implement the strategy method")
    
    def run_strategy(self):
        for i in range(0, len(self.data)-1):
            current_timestamp = self.data.iloc[i].timestamp

            # Execute strategy
            self.strategy(current_timestamp)
            
            # Check position value.
            current_close = self.get_current_prices(i).Close
            self.check_position_value(current_timestamp, current_close)
    """
    Orders
    """
    def check_position_value(self, current_timestamp):
        """
        Check if the total balance (including unrealized P&L from short positions) falls below minimum balance.
        
        Args:
            current_timestamp (int): Current timestamp in milliseconds
            current_close (float): Current closing price
        """
        # Calculate unrealized P&L differently for long and short positions
        current_close = self.get_current_prices(current_timestamp).Close
        if self.current_position >= 0:
            # Long position
            unrealized_pnl = self.current_position * current_close
        else:
            # Short position - profit when price goes down, loss when price goes up
            entry_price = self.get_average_entry_price()
            unrealized_pnl = (entry_price - current_close) * abs(self.current_position)
        
        total_balance = self.current_capital + unrealized_pnl
        
        if total_balance < self.minimum_balance:
            if self.verbose:
                print(f"WARNING: Balance ${round(total_balance, 2)} below minimum ${round(self.minimum_balance, 2)}")
                print(f"Current Capital: ${round(self.current_capital, 2)}")
                print(f"Unrealized P&L: ${round(unrealized_pnl, 2)}")
                print("Stopping trading and closing all positions")
            self.stop_trading = True
            self.close_current_positions(current_timestamp)

    def get_average_entry_price(self):
        """Calculate average entry price from recent trades"""
        if not self.trades:
            return None
            
        relevant_trades = [t for t in self.trades if (
            (t.side == OrderSide.BUY and self.current_position > 0) or
            (t.side == OrderSide.SELL and self.current_position < 0)
        )]
        
        if not relevant_trades:
            return None
            
        total_quantity = sum(t.quantity for t in relevant_trades)
        weighted_price = sum(t.price * t.quantity for t in relevant_trades)
        return weighted_price / total_quantity if total_quantity > 0 else None 
    def go_long(self, timestamp: int, quantity: float = None, amount: float = None):
        """
        Enter a long position, clearing any existing short position first.
        
        Args:
            timestamp (int): Current timestamp in milliseconds
            quantity (float, optional): Number of shares to buy
            amount (float, optional): Dollar amount to invest. If 'all', uses all available capital
        
        Returns:
            bool: True if all orders were executed successfully
        """
        success = True
        current_prices = self.get_current_prices(timestamp)
        next_open = self.data.iloc[current_prices.name + 1].Close
        
        # Clear existing short position if any
        self.close_current_positions(timestamp)

        # Calculate quantity if amount is specified
        if amount is not None:
            if amount == 'all':
                # Use all available capital (accounting for transaction costs)
                amount = self.current_capital / (1 + self.transaction_cost)
            quantity = int(amount / next_open)
        
        # Enter new long position if quantity is specified
        if quantity is not None and quantity > 0:
            success = success and self.execute_order(
                timestamp=timestamp,
                side=OrderSide.BUY,
                quantity=quantity
            )
            if self.verbose:
                print(f"Entered long position of {quantity} shares")
        
        return success

    def go_short(self, timestamp: int, quantity: float = None, amount: float = None):
        """
        Enter a short position, clearing any existing long position first.
        
        Args:
            timestamp (int): Current timestamp in milliseconds
            quantity (float, optional): Number of shares to sell short
            amount (float, optional): Dollar amount to short. If 'all', uses all available capital
        
        Returns:
            bool: True if all orders were executed successfully
        """
        success = True
        current_prices = self.get_current_prices(timestamp)
        next_open = self.data.iloc[current_prices.name + 1].Close
        
        # Clear existing long position if any
        self.close_current_positions(timestamp)
        
        # Calculate quantity if amount is specified
        if amount is not None:
            if amount == 'all':
                # Use all available capital (accounting for transaction costs)
                # For shorts, we need to be more conservative due to potential losses
                amount = self.current_capital / (1 + self.transaction_cost)
            quantity = int(amount / next_open)
        
        # Enter new short position if quantity is specified
        if quantity is not None and quantity > 0:
            success = success and self.execute_order(
                timestamp=timestamp,
                side=OrderSide.SELL,
                quantity=quantity
            )
            if self.verbose:
                print(f"Entered short position of {quantity} shares")
        
        return success

    def close_position(self, timestamp: int):
        """
        Close any existing position (long or short).
        
        Args:
            timestamp (int): Current timestamp in milliseconds
        
        Returns:
            bool: True if the position was closed successfully
        """
        if self.current_position == 0:
            return True
            
        if self.current_position > 0:
            return self.execute_order(
                timestamp=timestamp,
                side=OrderSide.SELL,
                quantity=self.current_position
            )
        else:
            return self.execute_order(
                timestamp=timestamp,
                side=OrderSide.BUY,
                quantity=abs(self.current_position)
            )
    def execute_order(self, timestamp: int, side: OrderSide, type: OrderType = OrderType.MARKET, 
                    quantity: float = None, amount: float = None):
        """
        Execute a trade order with simplified short position support. Orders are executed at the next timestamp's open price.
        
        Args:
            timestamp (int): Current timestamp in milliseconds
            side (OrderSide): BUY or SELL
            type (OrderType): Order type (currently only MARKET supported)
            quantity (float, optional): Number of shares to trade
            amount (float, optional): Dollar amount to trade
        
        Returns:
            bool: True if order was executed successfully
        """
        if self.stop_trading:
            if self.verbose:
                print("Trading has been stopped due to minimum balance breach")
            return False

        if quantity is None and amount is None:
            raise ValueError("Must specify either quantity or amount")

        current_prices = self.get_current_prices(timestamp)
        current_index = current_prices.name
        current_close = current_prices.Close
        next_prices = self.data.iloc[current_index + 1]
        next_open = next_prices.Close

        # Calculate quantity if amount is provided
        if amount is not None:
            quantity = int(amount / next_open)

        # Calculate transaction cost
        transaction_cost = self.transaction_cost * quantity * next_open

        if side == OrderSide.BUY:
            # Calculate total cost including transaction fees
            total_cost = (quantity * next_open) + transaction_cost
            
            # Check if we have enough capital
            if self.current_capital < total_cost:
                if self.verbose:
                    print(f"EXECUTE ORDER Error: Insufficient capital (${round(self.current_capital, 2)}) for trade costing ${round(total_cost, 2)}")
                return False
            
            # Execute buy order
            self.current_capital -= total_cost
            self.current_position += quantity
            
        elif side == OrderSide.SELL:
            # For short positions, check exposure against current known price
            short_exposure = quantity * current_close
            
            # Check if short exposure would exceed current capital
            if short_exposure > self.current_capital:
                if self.verbose:
                    print(f"EXECUTE ORDER Error: Short exposure (${round(short_exposure, 2)}) would exceed current capital (${round(self.current_capital, 2)})")
                return False
            
            # Execute sell order
            sale_proceeds = (quantity * next_open) - transaction_cost
            self.current_capital += sale_proceeds
            self.current_position -= quantity
            
        else:
            raise ValueError("Invalid OrderSide")

        # Record the trade
        trade = Trade(
            timestamp=next_prices.timestamp,
            side=side,
            type=type,
            quantity=quantity,
            price=next_open
        )
        self.trades.append(trade)

        if self.verbose:
            print(f"""Executed {side.value} order for {quantity} shares @ ${round(next_open, 2)}
                    Position: {self.current_position}
                    Capital: ${round(self.current_capital, 2)}""")
        return True

    def close_current_positions(self, timestamp: int):
        """
        Close all current positions at market price.
        
        Args:
            timestamp (int): Current timestamp in milliseconds
        
        Returns:
            bool: True if positions were closed successfully
        """
        if self.current_position == 0:
            return True
        if self.current_position > 0:
            return self.execute_order(
                timestamp=timestamp,
                side=OrderSide.SELL,
                type=OrderType.MARKET,
                quantity=self.current_position
            )
        else:
            return self.execute_order(
                timestamp=timestamp,
                side=OrderSide.BUY,
                type=OrderType.MARKET,
                quantity=abs(self.current_position)
            )

    """
    Helper functions
    """
    def realized_balance(self, current_timestamp):
        # prices = self.get_current_prices(current_timestamp)
        print(f"""Date : {pd.to_datetime(datetime.fromtimestamp(current_timestamp / 1000)).strftime("%d%m%Y %H:%M:%S")} UTC | Realized Balance : {self.current_capital}""")
        return self.current_capital
    def unrealized_balance(self, current_timestamp):
        prices = self.get_current_prices(current_timestamp)
        ub = self.current_position * prices.Close
        print(f"""Date : {pd.to_datetime(datetime.fromtimestamp(current_timestamp / 1000)).strftime("%d%m%Y %H:%M:%S")} UTC | Current Position : {self.current_position} | Unrealized Balance : ${round(ub, 2)}""")
        return ub
    def total_balance(self, current_timestamp):
        tb = self.realized_balance(current_timestamp) + self.unrealized_balance(current_timestamp)
        print(f"""Date : {pd.to_datetime(datetime.fromtimestamp(current_timestamp / 1000)).strftime("%d%m%Y %H:%M:%S")} UTC | Total Balance : ${round(tb, 2)}""")
        return tb
    
    def get_current_prices(self, timestamp):
        return self.data[self.data['timestamp'] == timestamp].iloc[0]
    
    def last_trade(self):
        if len(self.trades) == 0:
            return None
        return self.trades[-1]
    """
    Performance metrics
    """
    def calculate_performance(self, risk_free_rate = 0):
        """
        Calculate comprehensive performance metrics for the trading strategy.
        Returns a dictionary containing detailed performance metrics.
        """
        # Get daily positions and capital
        daily_data = []
        position_value = 0
        
        for i in range(len(self.data)):
            row = self.data.iloc[i]
            # Find all trades that happened on this day
            day_trades = [t for t in self.trades if t.timestamp <= row.timestamp]
            
            if day_trades:
                # Recalculate position based on trades
                position = sum(t.quantity if t.side == OrderSide.BUY else -t.quantity for t in day_trades)
                position_value = position * row.Close
            
            total_value = self.current_capital + position_value
            daily_data.append({
                'timestamp': row.timestamp,
                'datetime': pd.to_datetime(row.timestamp, unit='ms'),
                'total_value': total_value,
                'position': position_value if position_value else 0
            })
        
        df = pd.DataFrame(daily_data)
        df['daily_returns'] = df['total_value'].pct_change()
        df['daily_excess_returns'] = df['daily_returns'] - 0.02/252  # Daily risk-free rate
        
        # Basic metrics
        initial_capital = self.initial_capital
        final_capital = df['total_value'].iloc[-1]
        total_return = ((final_capital - initial_capital) / initial_capital) * 100
        
        # Time metrics
        days = (df['datetime'].iloc[-1] - df['datetime'].iloc[0]).days
        annual_factor = 252 / days
        # trading_days = len(df)
        
        # Returns and volatility metrics
        annual_return = ((1 + total_return/100) ** annual_factor - 1) * 100
        annual_std = df['daily_returns'].std() * np.sqrt(252) * 100
        
        # Risk metrics
        excess_returns = df['daily_excess_returns'].mean() * 252
        tracking_error = df['daily_excess_returns'].std() * np.sqrt(252)
        
        # Information ratio
        # information_ratio = excess_returns / tracking_error if tracking_error != 0 else 0
        
        # Sharpe and Sortino ratios
        sharpe_ratio = (annual_return/100 - risk_free_rate) / (annual_std/100)
        negative_returns = df['daily_returns'][df['daily_returns'] < 0]
        downside_std = negative_returns.std() * np.sqrt(252)
        sortino_ratio = (annual_return/100 - risk_free_rate) / downside_std if downside_std != 0 else 0
        
        # Drawdown analysis
        df['cummax'] = df['total_value'].cummax()
        df['drawdown'] = (df['cummax'] - df['total_value']) / df['cummax'] * 100
        max_drawdown = df['drawdown'].max()
        
        # Drawdown periods
        drawdown_periods = []
        current_period = 0
        for i in range(1, len(df)):
            if df['drawdown'].iloc[i] > 0:
                current_period += 1
            else:
                if current_period > 0:
                    drawdown_periods.append(current_period)
                current_period = 0
        if current_period > 0:
            drawdown_periods.append(current_period)
        max_drawdown_period = max(drawdown_periods) if drawdown_periods else 0
        
        # Trade analysis
        num_trades = len(self.trades)
        if num_trades > 0:
            profitable_trades = sum(1 for i in range(1, len(self.trades)) 
                                if (self.trades[i].side == OrderSide.SELL and self.trades[i].price > self.trades[i-1].price) or
                                    (self.trades[i].side == OrderSide.BUY and self.trades[i].price < self.trades[i-1].price))
            win_rate = (profitable_trades / num_trades) * 100
            
            # Calculate average trade metrics
            trade_returns = []
            for i in range(1, len(self.trades), 2):  # Assuming trades come in pairs
                if i < len(self.trades):
                    entry = self.trades[i-1]
                    exit = self.trades[i]
                    if entry.side == OrderSide.BUY:
                        trade_return = (exit.price - entry.price) / entry.price
                    else:
                        trade_return = (entry.price - exit.price) / entry.price
                    trade_returns.append(trade_return)
            
            avg_trade_return = np.mean(trade_returns) * 100 if trade_returns else 0
            max_trade_return = max(trade_returns) * 100 if trade_returns else 0
            min_trade_return = min(trade_returns) * 100 if trade_returns else 0
        else:
            win_rate = 0
            avg_trade_return = 0
            max_trade_return = 0
            min_trade_return = 0
        
        return {
            # Original metrics
            'final_capital': round(final_capital, 2),
            'total_return': round(total_return, 2),
            'annual_return': round(annual_return, 2),
            'annual_std': round(annual_std, 2),
            'sharpe_ratio': round(sharpe_ratio, 2),
            'sortino_ratio': round(sortino_ratio, 2),
            'max_drawdown': round(max_drawdown, 2),
            'max_drawdown_period': round(max_drawdown_period, 2),
            
            # New trade metrics
            'number_of_trades': num_trades,
            'win_rate': round(win_rate, 2),
            'avg_trade_return': round(avg_trade_return, 2),
            'max_trade_return': round(max_trade_return, 2),
            'min_trade_return': round(min_trade_return, 2),
        }
    """
    Plotters
    """
    def plot_performance(self):
        """
        Creates a comprehensive visualization of strategy performance metrics.
        Includes returns, drawdowns, distributions, and equity curves.
        """
        # Get performance data
        metrics, df = self.calculate_performance()
        
        # Calculate additional metrics for plotting
        df['log_returns'] = np.log(df['total_value'] / df['total_value'].shift(1))
        df['cum_returns'] = df['log_returns'].cumsum()
        df['rolling_max'] = df['total_value'].expanding().max()
        df['drawdown'] = (df['rolling_max'] - df['total_value']) / df['rolling_max'] * 100
        
        # Create subplots
        fig, axs = plt.subplots(2, 3, figsize=(20, 10))
        plt.subplots_adjust(hspace=0.3, wspace=0.3)
        
        # 1. Cumulative Returns Plot
        axs[0, 0].plot(df['datetime'], df['cum_returns'], label='Strategy Returns', color='blue')
        axs[0, 0].set_title('Cumulative Log Returns')
        axs[0, 0].set_xlabel('Date')
        axs[0, 0].set_ylabel('Cumulative Returns')
        axs[0, 0].grid(True)
        axs[0, 0].legend()
        
        # 2. Returns Distribution
        sns.histplot(df['log_returns'].dropna() * 100, bins=50, ax=axs[0, 1])
        axs[0, 1].set_title('Returns Distribution')
        axs[0, 1].set_xlabel('Daily Returns (%)')
        axs[0, 1].set_ylabel('Frequency')
        
        # Add normal distribution overlay
        returns_mean = df['log_returns'].mean()
        returns_std = df['log_returns'].std()
        x = np.linspace(df['log_returns'].min(), df['log_returns'].max(), 100)
        y = stats.norm.pdf(x, returns_mean, returns_std)
        scaled_y = y * (df['log_returns'].count() * (df['log_returns'].max() - df['log_returns'].min()) / 50)
        axs[0, 1].plot(x * 100, scaled_y, 'r--', label='Normal Dist.')
        axs[0, 1].legend()
        
        # 3. Strategy vs Benchmark (using close price as basic benchmark)
        benchmark_returns = np.log(self.data['Close'] / self.data['Close'].shift(1)).cumsum()
        axs[0, 2].plot(df['datetime'], df['cum_returns'], label='Strategy', color='blue')
        axs[0, 2].plot(df['datetime'], benchmark_returns, label='Buy & Hold', color='red', linestyle='--')
        axs[0, 2].set_title('Strategy vs Buy & Hold')
        axs[0, 2].set_xlabel('Date')
        axs[0, 2].set_ylabel('Cumulative Returns')
        axs[0, 2].grid(True)
        axs[0, 2].legend()
        
        # 4. Drawdown
        axs[1, 0].fill_between(df['datetime'], df['drawdown'], 0, color='red', alpha=0.3)
        axs[1, 0].set_title('Drawdown')
        axs[1, 0].set_xlabel('Date')
        axs[1, 0].set_ylabel('Drawdown (%)')
        axs[1, 0].grid(True)
        
        # 5. Equity Curve
        axs[1, 1].plot(df['datetime'], df['total_value'], label='Portfolio Value', color='green')
        axs[1, 1].set_title('Equity Curve')
        axs[1, 1].set_xlabel('Date')
        axs[1, 1].set_ylabel('Portfolio Value ($)')
        axs[1, 1].grid(True)
        axs[1, 1].legend()
        
        # Remove the last subplot
        fig.delaxes(axs[1, 2])
        
        # Add performance metrics as text
        metrics_text = (
            f"Total Return: {metrics['total_return']:.1f}%\n"
            f"Annual Return: {metrics['annual_return']:.1f}%\n"
            f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}\n"
            f"Max Drawdown: {metrics['max_drawdown']:.1f}%\n"
            f"Win Rate: {metrics['win_rate']:.1f}%\n"
            f"Profit Factor: {metrics['profit_factor']:.2f}"
        )
        fig.text(0.75, 0.35, metrics_text, fontsize=10, bbox=dict(facecolor='white', alpha=0.8))
        
        plt.suptitle('Strategy Performance Analysis', fontsize=16, y=1.02)
        plt.tight_layout()
        plt.show()

    def plot_close(self):
        """
        Plots the close price with buy/sell signals and position sizes.
        """
        # Create figure and axis
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), gridspec_kw={'height_ratios': [3, 1]})
        
        # Plot close price
        ax1.plot(self.data['datetime'], self.data['Close'], label='Close Price', color='blue', alpha=0.7)
        
        # Add buy/sell markers
        for trade in self.trades:
            if trade.side == OrderSide.BUY:
                ax1.scatter(pd.to_datetime(trade.timestamp, unit='ms'), 
                        trade.price, 
                        color='green', 
                        marker='^', 
                        s=100, 
                        label='Buy' if 'Buy' not in ax1.get_legend_handles_labels()[1] else "")
            else:
                ax1.scatter(pd.to_datetime(trade.timestamp, unit='ms'), 
                        trade.price, 
                        color='red', 
                        marker='v', 
                        s=100, 
                        label='Sell' if 'Sell' not in ax1.get_legend_handles_labels()[1] else "")
        
        # Calculate position sizes for each timestamp
        positions = []
        current_pos = 0
        for _, row in self.data.iterrows():
            trades_at_time = [t for t in self.trades if t.timestamp <= row['timestamp']]
            if trades_at_time:
                current_pos = sum(t.quantity if t.side == OrderSide.BUY else -t.quantity for t in trades_at_time)
            positions.append(current_pos)
        
        # Plot position size
        ax2.fill_between(self.data['datetime'], positions, 0, 
                        where=(np.array(positions) > 0), color='green', alpha=0.3, label='Long Position')
        ax2.fill_between(self.data['datetime'], positions, 0, 
                        where=(np.array(positions) < 0), color='red', alpha=0.3, label='Short Position')
        
        # Customize plots
        ax1.set_title('Price Chart with Trading Signals')
        ax1.set_ylabel('Price')
        ax1.grid(True)
        ax1.legend()
        
        ax2.set_title('Position Size')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Position Size')
        ax2.grid(True)
        ax2.legend()
        
        plt.tight_layout()
        plt.show()

In [2]:
import yfinance as yf

stock = yf.Ticker("MSFT")
historical_data = stock.history(start = "2010-01-01", end = "2021-01-01", interval = "1d").reset_index().rename(columns = {'Open' : 'open', 'High' : 'high', 'Low' : 'low', 'Close' : 'close', 'Volume' : 'volume'})
historical_data['timestamp'] = historical_data['Date'].apply(lambda x : int(x.timestamp() * 1000)) 
input_data = historical_data[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
input_data

,timestamp,open,high,low,close,volume
0,1262581200000,23.098381,23.460472,23.075750,23.347319,38409100
1,1262667600000,23.271890,23.460479,23.113474,23.354868,49749600
2,1262754000000,23.294513,23.445384,23.022945,23.211535,58182400
3,1262840400000,23.105925,23.158731,22.774010,22.970142,50559700
4,1262926800000,22.841899,23.294511,22.811724,23.128553,51197400
...,...,...,...,...,...,...
2764,1608786000000,214.456971,216.578104,214.243888,215.745148,10550600
2765,1609131600000,217.391725,218.922040,216.006701,217.885696,17933500
2766,1609218000000,219.193213,220.035849,216.549068,217.101135,17403200
2767,1609304400000,218.147167,218.534597,214.505413,214.708801,20272300


In [5]:
class MAStrategy(EventBasedBacktester):
    def __init__(self, symbol, start, end, interval, capital, transaction_cost, verbose=True):
        super().__init__(symbol, start, end, interval, capital, transaction_cost, verbose)
        
    def prepare_indicators(self, short_ma_window=50, long_ma_window=200):
        """Calculate moving averages and trading signals"""
        self.short_ma_window = short_ma_window
        self.long_ma_window = long_ma_window
        
        # Calculate MAs
        self.data['short_ma'] = self.data['Close'].rolling(window=short_ma_window).mean()
        self.data['long_ma'] = self.data['Close'].rolling(window=long_ma_window).mean()
        
        # Calculate crossover signals
        self.data['ma_diff'] = self.data['short_ma'] - self.data['long_ma']
        self.data['signal'] = np.where(self.data['ma_diff'] > 0, 1,
                                     np.where(self.data['ma_diff'] < 0, -1, 0))
        
        # Calculate signal changes
        self.data['signal_change'] = self.data['signal'].diff()
    
    def strategy(self, timestamp):
        if self.stop_trading:
            return
            
        current_data = self.get_current_prices(timestamp)
        current_index = current_data.name
        
        if current_index <= self.long_ma_window:
            return
            
        signal_change = self.data.iloc[current_index]['signal_change']
        
        if signal_change == 0:
            return
            
        if signal_change > 0:
            if self.verbose:
                print(f"\nMA Crossover Signal: LONG at {pd.to_datetime(timestamp, unit='ms')}")
                print(f"Short MA: {self.data.iloc[current_index]['short_ma']:.2f}")
                print(f"Long MA: {self.data.iloc[current_index]['long_ma']:.2f}")
            self.go_long(timestamp, amount='all')
        elif signal_change < 0:
            if self.verbose:
                print(f"\nMA Crossover Signal: SHORT at {pd.to_datetime(timestamp, unit='ms')}")
                print(f"Short MA: {self.data.iloc[current_index]['short_ma']:.2f}")
                print(f"Long MA: {self.data.iloc[current_index]['long_ma']:.2f}")
            self.go_short(timestamp, amount='all')

In [6]:
ma_strategy_1 = MAStrategy(
    symbol="AAPL",
    start="2010-01-01",
    end="2021-01-01",
    interval = "1d",
    capital=10000,
    transaction_cost=0,
)
ma_strategy_1.prepare_indicators(short_ma_window=50, long_ma_window=200)
ma_strategy_1.run_strategy()

IndexError: single positional indexer is out-of-bounds

## 2 EventBasedPortfolioManager - A class that takes in a list of EventBasedBacktesters, runs the strategies and calculates the peformance of the portfolio.

1. List of EventBasedBacktester
2. Weights
3. Risk Management
   1. contribution of each strategy
4. Portfolio Analytics 
   1. Calculate portoflio level total expos
   2. portfolio report